In [1]:
import sys, importlib, glob
import ncbi_crassvirales as ncbi_crassvirales
import utils as utils
import sister_clades as sister_clades
import terl_tree as terl_tree
from ete3 import RectFace, faces, Tree
import pandas as pd

/home/dani/miniconda3/lib/python3.9/site-packages/ete3-3.1.2-py3.7.egg/ete3/evol/parser/codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?
/home/dani/miniconda3/lib/python3.9/site-packages/ete3-3.1.2-py3.7.egg/ete3/evol/parser/codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?


In [3]:
# get crassvirales taxonomy
#taxa_file = "/home/danielc/software/github/old_june/tree_functions/files/crassphages_taxonomy_terL_and_new.txt"
taxa_file = "crassphages_taxonomy_terL_and_new.txt"
tax_df = terl_tree.get_taxonomy_df(taxa_file)

In [4]:
# parse TerL tree
importlib.reload(terl_tree);
tree_file = "/home/danielc/projects/Bas_phages/ani_crass_genomes/iqtree/terl_ref_distant_fftns1000_trim09.treefile"
tree_file = "/home/dani/encode_home/projects/Bas_phages/ani_crass_genomes/iqtree/terl_ref_distant_fftns1000_trim09.treefile"
tree_file = "/home/danielc/projects/Bas_phages/terL_sequences_trimmed_merged_10gaps.treefile"
tree_file = "/home/dani/encode_home/projects/Bas_phages/terL_sequences_trimmed_merged_10gaps.treefile"
t, ts = terl_tree.parse_terl_tree(tree_file, tax_df)
terl_genomes_seqs = sister_clades.terl_genomes_seqs(t)
# add two features to the inner nodes: superkingdoms and phyla
# add them also to the leaves
for node in t.traverse():
    #if not node.is_leaf():
    node.add_features(superkingdoms=dict(),
                      phyla=dict(),
                      order=dict()
                     )

#t.show()
t.write(outfile=f"{tree_file}.rooted", format=1)

In [10]:
t.show()

In [11]:
### list sister clades files
importlib.reload(sister_clades);
sister_clades_dir = "/home/dani/encode_home/projects/Bas_phages/5_nr_screening/4_merged_ncbi_crassvirales/4_MRCAs_taxa"
sister_clades_tables = glob.glob(f"{sister_clades_dir}/*txt")

for table in sister_clades_tables:
    df = sister_clades.parse_sister_clades_table(table)
    if df.shape[0] > 1:
        cl_id = df["cl_id"][0]
        # iterate the crass clades
        for crass_clade in df.index:
            clade_seqs_names = df.loc[crass_clade, "seqs_mrca"].split(",")
            # choose which rank to show
            phyla_sister = df.loc[crass_clade, "bact_phyla_sister"]
            order_sister = df.loc[crass_clade, "bact_order_sister"]


            t = sister_clades.process_crass_clade(cl_id, clade_seqs_names, terl_genomes_seqs, t, phyla_sister)


cl_s_253 already present in node104
	-Taxa before: {'Proteobacteria': 1.0}
	-Taxa to add: {'Firmicutes': 1.0}
	-Updated: {'Firmicutes': 0.5, 'Proteobacteria': 0.5}
cl_s_190 already present in node186
	-Taxa before: {'Bacteroidetes': 1.0}
	-Taxa to add: {'Bacteroidetes': 0.974, 'Firmicutes': 0.026}
	-Updated: {'Bacteroidetes': 0.987, 'Firmicutes': 0.013}
cl_1099 already present in node42
	-Taxa before: {'Firmicutes': 1.0}
	-Taxa to add: {'Firmicutes': 1.0}
	-Updated: {'Firmicutes': 1.0}
cl_s_658 already present in node111
	-Taxa before: {'Bacteroidetes': 1.0}
	-Taxa to add: {'Bacteroidetes': 1.0}
	-Updated: {'Bacteroidetes': 1.0}
cl_0217 already present in node878
	-Taxa before: {'Bacteroidetes': 1.0}
	-Taxa to add: {'Bacteroidetes': 1.0}
	-Updated: {'Bacteroidetes': 1.0}
cl_0217 already present in node878
	-Taxa before: {'Bacteroidetes': 1.0}
	-Taxa to add: {'Firmicutes': 1.0}
	-Updated: {'Firmicutes': 0.5, 'Bacteroidetes': 0.5}
cl_0217 already present in node878
	-Taxa before: {'Firmi

In [14]:
t.show()

In [12]:
importlib.reload(sister_clades);
t = sister_clades.phyla_terl_nodes(t)
t = sister_clades.order_terl_nodes(t)

In [13]:
importlib.reload(sister_clades);
itol_annot_phyla, phyla = sister_clades.get_itol_annotation_phyla(t)
itol_annot_order, orders = sister_clades.get_itol_annotation_order(t)

In [23]:
importlib.reload(sister_clades);
template = "../../dataset_piechart_template.txt"
outfile = "../../phyla_annot_piecharts.txt"
#outfile = "../../order_annot_piecharts.txt"
sister_clades.write_itol_dataset(itol_annot_phyla, phyla, template, outfile)

In [17]:
importlib.reload(sister_clades);
outfile = "../../terl_ranges.txt"
sister_clades.write_itol_taxa_annot_terl(tax_df, t, outfile)

Intestiviridae
Crevaviridae
Suoliviridae
Steigviridae
Tinaiviridae
Jelitoviridae


In [18]:
for node in t.traverse():
    if not node.is_leaf():
        if "cl_0128" in node.phyla:
            print(node.name)

node24
node252
node314
node532
node791


In [13]:
t.show()

In [37]:
terl_genomes_seqs["OHFX01000006"]

'OHFX01000006|791|208_210'

In [38]:
t.get_common_ancestor("OHFX01000006|791|208_210")

Tree node 'node1' (0x7f930496994)

In [54]:
d1 =  {"1": ["contig1", "contig2", "contig3"], "2":["contig4", "contig5"]}
d2 = {"contig1": "bacteria", "contig2": "bacteria", "contig3": "virus", "contig4": "virus", "contig5": "bacteria"}

In [55]:
d3 = dict()
for key, contigs in d1.items():
    d3[key] = dict()
    for contig in contigs:
        d3[key][contig] = d2[contig]

In [17]:
node112 = t.search_nodes(name="node112")[0]
node112.show()

In [19]:
node112.cl

'cl_0929,cl_s_476,cl_1676,cl_1022,cl_0972,cl_1828,cl_s_556,cl_0991,cl_0900,cl_2122,cl_2461,cl_0990,cl_0927,cl_1241,cl_s_731,cl_s_469,cl_1854,cl_1647,cl_0968,cl_s_744,cl_0964,cl_1907,cl_0858,cl_s_704,cl_2405,cl_0958,cl_s_531,cl_s_218,cl_1857,cl_s_678,cl_1917,cl_s_548,cl_s_353,cl_1768,cl_0954,cl_1086,cl_1003,cl_s_472,cl_s_666,cl_0815,cl_1417,cl_s_122,cl_2215,cl_0981,cl_1748'